In [4]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)
    
# 3_testowanie_modeli.py
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from joblib import load
import tensorflow as tf
import torch

# Wczytanie danych
data = pd.read_excel("złączone_dane.xlsx")
data = data.drop("image_id", axis=1)
data = data.drop(columns=[col for col in data.columns if any(x in col for x in ['3_p', '4_p', '5_p'])])

X = data.drop('label', axis=1)
y = data['label']
le = LabelEncoder()
y_encoded = le.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Funkcja oceniająca
def evaluate_model(model, name):
    y_pred = model.predict(X_scaled)
    print(f"\n📌 Model: {name}")
    print(classification_report(y_encoded, y_pred, digits=4, target_names=le.classes_))


# Klasyczne modele
models = ["RandomForest", "LogisticRegression", "KNN", "SVM", "NaiveBayes", "DecisionTree", "MLP"]
for name in models:
    model = load(f"models/{name}.pkl")
    evaluate_model(model, name)

# TensorFlow
model_tf = tf.keras.models.load_model("models/NeuralNet_TF.h5")
y_pred_tf = np.argmax(model_tf.predict(X_scaled), axis=1)
evaluate_model(type('TFWrapper', (object,), {"predict": lambda self, _: y_pred_tf})(), "NeuralNet_TF")

# PyTorch
model_pt = Net(X_scaled.shape[1], len(np.unique(y_encoded)))
model_pt.load_state_dict(torch.load("models/NeuralNet_PT.pt"))
model_pt.eval()
with torch.no_grad():
    y_pred_pt = model_pt(torch.tensor(X_scaled, dtype=torch.float32))
    y_pred_pt = torch.argmax(y_pred_pt, axis=1).numpy()
evaluate_model(type('PTWrapper', (object,), {"predict": lambda self, _: y_pred_pt})(), "NeuralNet_PT")



📌 Model: RandomForest
              precision    recall  f1-score   support

           a     1.0000    1.0000    1.0000       305
          a+     1.0000    1.0000    1.0000       526
           b     1.0000    1.0000    1.0000       303
           c     1.0000    1.0000    1.0000        59
          c+     1.0000    1.0000    1.0000       388
          ch     1.0000    1.0000    1.0000       521
          cz     1.0000    1.0000    1.0000       530
           d     1.0000    1.0000    1.0000       450
           e     1.0000    1.0000    1.0000        41
          e+     1.0000    1.0000    1.0000       439
           f     1.0000    1.0000    1.0000       573
           g     1.0000    1.0000    1.0000       599
           h     1.0000    1.0000    1.0000       529
           i     1.0000    1.0000    1.0000        61
           j     1.0000    1.0000    1.0000       452
           k     1.0000    1.0000    1.0000       525
           l     1.0000    1.0000    1.0000       343
    


📌 Model: NaiveBayes
              precision    recall  f1-score   support

           a     0.9722    0.9180    0.9444       305
          a+     0.8814    0.9468    0.9129       526
           b     0.9481    0.9637    0.9558       303
           c     0.5783    0.8136    0.6761        59
          c+     0.8483    0.7062    0.7707       388
          ch     0.6751    0.5624    0.6136       521
          cz     0.9124    0.6094    0.7308       530
           d     0.8972    0.7956    0.8433       450
           e     0.7857    0.8049    0.7952        41
          e+     0.8952    0.8565    0.8754       439
           f     0.8506    0.9040    0.8765       573
           g     0.9684    0.8698    0.9164       599
           h     0.5691    0.7316    0.6402       529
           i     0.8281    0.8689    0.8480        61
           j     0.8504    0.8805    0.8652       452
           k     0.9979    0.9124    0.9532       525
           l     0.9969    0.9446    0.9701       343
      